#### Preprocess the dataset and remove the outliers to keep the data in same range

1. Get the before and after code from the dataset and get the magic number

In [1]:
import analyze_code

num_lines_around_declaration = 1
num_lines_above_usage = 5
num_lines_below_usage = 5

In [6]:
import analyze_code

num_lines_around_declaration = 1
num_lines_above_usage = 3
num_lines_below_usage = 3

input_code_java = """
    public class NetworkStatsService extends INetworkStatsService.Stub {
        private static final int FILE_MAGIC = 0x414E4554;

        private void readStatsLocked() {
            if (LOGD) Slog.v(TAG, "readStatsLocked()");

            // clear any existing stats and read from disk
            mSummaryStats.clear();

            FileInputStream fis = null;
            try {
                fis = mSummaryFile.openRead();
                final DataInputStream in = new DataInputStream(fis);

                // verify file magic header intact
                final int magic = in.readInt();
                if (magic != FILE_MAGIC) {
                    throw new ProtocolException("unexpected magic: " + magic);
                }

                final int version = in.readInt();
                switch (version) {
                    case 100: {
                        // file format is pairs of interfaces and stats:
                        // summary := size *(InterfaceIdentity NetworkStatsHistory)

                        final int size = in.readInt();
                        for (int i = 0; i < size; i++) {
                            final InterfaceIdentity ident = new InterfaceIdentity(in);
                            final NetworkStatsHistory history = new NetworkStatsHistory(in);
                            mSummaryStats.put(ident, history);
                        }
                        break;
                    }
                    default: {
                        throw new ProtocolException("unexpected version: " + version);
                    }
                }
            } catch (FileNotFoundException e) {
                // missing stats is okay, probably first boot
            } catch (IOException e) {
                Slog.e(TAG, "problem reading network stats", e);
            } finally {
                IoUtils.closeQuietly(fis);
            }
        }
    }"""
# method_code = analyze_code.find_method(input_code_java, 111)

declaration_line, selected_code = analyze_code.extract_code_around_constant_declaration(
    input_code_java, num_lines_around_declaration, num_lines_above_usage, num_lines_below_usage
)
# print("method_code", method_code)

if declaration_line is not None and selected_code is not None:
    print("Constant Declaration Line:")
    print(declaration_line)
    print("\nLines Around Constant Usage:")
    print(selected_code)
else:
    print("No constant declarations found.")

Constant Declaration Line:
        private static final int FILE_MAGIC = 0x414E4554;

Lines Around Constant Usage:

                // verify file magic header intact
                final int magic = in.readInt();
                if (magic != FILE_MAGIC) {
                    throw new ProtocolException("unexpected magic: " + magic);
                }



In [4]:
dataset = analyze_code.read_jsonl("data/dataset_v2.jsonl")
total_count = 0
with_none = 0
rec_count = 0
for result in dataset:
    total_count += 1
    declaration_line, selected_code = analyze_code.extract_code_around_constant_declaration(
        result['after'], num_lines_around_declaration, num_lines_above_usage, num_lines_below_usage
    )
    if declaration_line is not None and selected_code is not None:
        rec_count += 1
        # if declaration_line is not None and selected_code is not None:
        #     print("Constant Declaration Line:")
        #     print(declaration_line)
        #     print("\nLines Around Constant Usage:")
        # print(selected_code)
        # else:
        #     print("No constant declarations found.")
        # print("result: ", result)
        final = analyze_code.extract_constant_info(declaration_line)
        # print("final is: ", final)
        # print(final['constant_name'])
        # print(final['constant_value'])
        # print(final['constant_value'])
        if final is not None and final['constant_value'] != 'null':
            updated_code = analyze_code.replace_constant_with_value(final['constant_name'], final['constant_value'], selected_code)
            # print(updated_code)
            analyze_code.append_fields_to_jsonl(updated_code, declaration_line+"\n"+selected_code, "data/analyzed_dataset_5_lines.jsonl")

print("total: ", total_count)
print("with_none: ", with_none)
print("rec: ", rec_count)

total:  4514
with_none:  0
rec:  4396


In [2]:
dataset = analyze_code.read_jsonl("data/dataset_v1.jsonl")
total_count = 0
with_none = 0
rec_count = 0
for result in dataset:
    total_count += 1
    declaration_line, selected_code, method_code = analyze_code.extract_code_around_constant_declaration(
        result['after'], num_lines_around_declaration, num_lines_above_usage, num_lines_below_usage
    )
    if declaration_line is not None and selected_code is not None and method_code is None:
        with_none += 1
    if declaration_line is not None and selected_code is not None and method_code is not None:
        rec_count += 1
        # if declaration_line is not None and selected_code is not None:
        #     print("Constant Declaration Line:")
        #     print(declaration_line)
        #     print("\nLines Around Constant Usage:")
        # print(selected_code)
        # else:
        #     print("No constant declarations found.")
        # print("result: ", result)
        final = analyze_code.extract_constant_info(declaration_line)
        # print("final is: ", final)
        # print(final['constant_name'])
        # print(final['constant_value'])
        # print(final['constant_value'])
        if final is not None and final['constant_value'] != 'null':
            updated_code, updated_method_code = analyze_code.replace_constant_with_value(final['constant_name'], final['constant_value'], selected_code, method_code)
            # print(updated_code)
            analyze_code.append_fields_to_jsonl(updated_code, declaration_line+"\n"+selected_code, updated_method_code, method_code,  "data/analyzed_dataset_4900.jsonl")

print("total: ", total_count)
print("with_none: ", with_none)
print("rec: ", rec_count)

ValueError: not enough values to unpack (expected 3, got 2)

In [11]:
final = analyze_code.extract_constant_info('public static final float str = "test0";')
print(final)

{'Constant Name': 'str', 'Constant Value': '"test0"'}
